In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from data_proccessing import train_loader, valid_loader,train_loader, labels

d:\adl\data_proccessing.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_new, data_entry[data_entry[l]==1][:500]], ignore_index=True)


In [2]:
alexnet = models.alexnet(pretrained=True)

# Freeze the weights of the pre-trained layers
for param in alexnet.parameters():
    param.requires_grad = False

# Unfreeze the weights of the last layer
for param in alexnet.classifier.parameters():
    param.requires_grad = True

# Modify the last fully connected layer
in_features = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=len(labels))

# Move the model to the appropriate device
device = 'cpu'
alexnet.to(device)

# Define the optimizer only for the parameters that require gradient calculation
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, alexnet.parameters()), lr=0.0001)

c:\Users\mandi\anaconda3\envs\adl\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mandi\anaconda3\envs\adl\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        # Move data to device
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = alexnet(images)
        # labels = labels.view(-1)
        # print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Display
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, len(train_loader), total_loss / 10))
            total_loss = 0.0

KeyboardInterrupt: 